In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load a pre-trained VGG model
vgg_model = models.vgg16(pretrained=True).to(device)

# We will use the features up to the second max pooling layer of the VGG model
# Create a new Sequential module containing the desired layers
vgg_feature_extractor = nn.Sequential(*list(vgg_model.features)[:9]).to(device)
# Calculate the correct input dimensions for the first fully connected layer after the VGG feature extractor
with torch.no_grad():
    # Use the same dummy input size as your training images
    dummy_input = torch.randn(1, 3, 32, 32, device=device)  # Example for CIFAR-10
    dummy_output = vgg_feature_extractor(dummy_input)
    # Flatten the output to calculate the total number of flat features
    flat_features = int(torch.flatten(dummy_output).shape[0])

# Set the VGG model to evaluation mode
vgg_feature_extractor.eval()

# Freeze the parameters of the feature extractor to prevent training
for param in vgg_feature_extractor.parameters():
    param.requires_grad = False

# Simple Convolutional Neural Network Model Definition
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.num_classes = num_classes  # Added this attribute
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class TRADESCNN(SimpleCNN):
    def __init__(self, num_classes, beta):
        super(TRADESCNN, self).__init__(num_classes)
        self.beta = beta

    # Implement the TRADES adversarial training within the loss function
    def forward(self, x):
        return super(TRADESCNN, self).forward(x)

    def calculate_loss(self, model_output, target, optimizer, x_natural, x_adv):
        criterion_natural = nn.CrossEntropyLoss()
        criterion_robust = nn.KLDivLoss(reduction='batchmean')
        loss_natural = criterion_natural(model_output, target)
        loss_robust = criterion_robust(F.log_softmax(x_adv, dim=1), F.softmax(x_natural, dim=1))
        loss = loss_natural + self.beta * loss_robust
        return loss

# Perceptual Adversarial Training (PAT) Model Definition
class PATCNN(SimpleCNN):
    def __init__(self, num_classes=10):
        super(PATCNN, self).__init__(num_classes=num_classes)
        self.feature_extractor = vgg_feature_extractor
        self.fc1 = nn.Linear(flat_features, 120)


    def forward(self, x):
        # Extract perceptual features and pass them through the rest of the network
        perceptual_features = self.feature_extractor(x)
        x = perceptual_features.view(perceptual_features.size(0), -1)  # Flatten the features
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x, perceptual_features

# RoBal Model Definition
class RoBalCNN(SimpleCNN):
    def __init__(self, num_classes=10, dataset=None):
        super(RoBalCNN, self).__init__(num_classes=num_classes)
        self.class_weights = self.calculate_class_weights(dataset)

    def calculate_class_weights(self, dataset):
        # This function will calculate the weight for each class based on the
        # inverse frequency of the classes in the dataset provided.
        # If dataset is None or not provided, default equal weights are used.
        if dataset is None:
            return torch.ones(self.num_classes)

        class_counts = torch.zeros(self.num_classes)
        for _, target in dataset:
            class_counts[target] += 1

        # Use inverse frequency to determine class weights
        # Adding 1 to avoid division by zero for classes not present in the dataset
        class_weights = 1.0 / (class_counts + 1.0)

        # Normalize weights such that they sum to the number of classes
        class_weights = self.num_classes * class_weights / class_weights.sum()
        return class_weights.to(device)

    def forward(self, x):
        return super(RoBalCNN, self).forward(x)

    def compute_loss(self, outputs, targets):
        # This function is an example where the loss function is given
        # additional class weights computed from the dataset. It should
        # be used in place of the typical criterion during training.
        return F.cross_entropy(outputs, targets, weight=self.class_weights)




/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 184MB/s]


In [3]:
# Dataset and DataLoader setup
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

batch_size = 4

trainloader_cifar10 = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform),
    batch_size=batch_size, shuffle=True, num_workers=2)

testloader_cifar10 = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform),
    batch_size=batch_size, shuffle=False, num_workers=2)

trainloader_cifar100 = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform),
    batch_size=batch_size, shuffle=True, num_workers=2)

testloader_cifar100 = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform),
    batch_size=batch_size, shuffle=False, num_workers=2)

# ImageNet dataset is large and not directly accessible; the code below is a placeholder
# trainloader_imagenet = torch.utils.data.DataLoader(...)
# testloader_imagenet = torch.utils.data.DataLoader(...)

# Training and testing on CIFAR-10 for SimpleCNN
net_cifar10 = SimpleCNN(num_classes=10).to(device)
# Training and testing on CIFAR-100 for SimpleCNN
net_cifar100 = SimpleCNN(num_classes=100).to(device)
# Assuming there are 10 classes for the custom model and the `beta` is initialized with a random example value of 1.0.
net_trades_cifar10 = TRADESCNN(num_classes=10, beta=1.0).to(device)
# Instantiate the PATCNN model as well.
net_pat_cifar10 = PATCNN(num_classes=10).to(device)
# Instantiate the RoBalCNN model for CIFAR-10.
net_robal_cifar10 = RoBalCNN(num_classes=10, dataset=trainloader_cifar10.dataset).to(device)


# Define a function for training
def train(net, trainloader, epochs):
    net.to(device)
    net.train()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:
                print(f'[Epoch {epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

# Define a function for testing
# Define a function for testing
def test(net, testloader):
    net.to(device)
    net.eval()

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            # If the network's forward function returns multiple outputs, we take the first one
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

def evaluate_robustness(net, testloader, epsilon=0.05):
    net.to(device)
    net.eval()

    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # Add a small perturbation to the images
        perturbed_images = images + epsilon * torch.randn_like(images)
        perturbed_images = torch.clamp(perturbed_images, 0, 1)  # Ensure it's still a valid image
        outputs = net(perturbed_images)
        if isinstance(outputs, tuple):
                outputs = outputs[0]
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    robustness_accuracy = correct / total
    return robustness_accuracy

def fgsm_attack(image, epsilon, data_grad):
    """
    Carries out FGSM attack to create adversarial examples.

    Args:
    image (Tensor): Original image.
    epsilon (float): Step size (perturbation amount).
    data_grad (Tensor): Gradient of the loss with respect to the input image.

    Returns:
    perturbed_image (Tensor): Adversarial image.
    """
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon * sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image


def train_with_adversarial(net, trainloader, epsilon, epochs):
    """
    Trains a neural network with adversarial examples using FGSM.

    Args:
    net (nn.Module): Neural network model.
    trainloader (DataLoader): DataLoader for training data.
    epsilon (float): Strength of the adversarial perturbation.
    epochs (int): Number of epochs to train for.
    """
    net.train()  # Make sure the network is in training mode

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(epochs):
        for i, (inputs, labels) in enumerate(trainloader, 0):
            inputs, labels = inputs.to(device), labels.to(device)

            # Tell PyTorch to track gradients for the inputs
            inputs.requires_grad = True

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass to calculate predictions and loss
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # Get the gradients for the input data to generate adversarial examples
            data_grad = inputs.grad.data

            # Create adversarial examples using the FGSM method
            perturbed_data = fgsm_attack(inputs, epsilon, data_grad)

            # Re-classify the perturbed images
            outputs_perturbed = net(perturbed_data)
            loss_perturbed = criterion(outputs_perturbed, labels)

            # Zero the parameter gradients again before the backward pass
            optimizer.zero_grad()

            # Calculate gradients for the perturbed data
            loss_perturbed.backward()

            # Update the weights
            optimizer.step()

            # Print statistics
            if i % 1000 == 999:  # Print every 1000 mini-batches
                print(f'[Epoch {epoch + 1}, Mini-batch {i + 1}] loss: {loss.item():.3f}')



Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
def train_TRADES(net, trainloader, epsilon, beta, epochs):
    net.train()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Tell PyTorch to track gradients for the inputs
            inputs.requires_grad_(True)

            # Forward pass to compute the loss on clean inputs
            outputs_clean = net(inputs)
            loss_clean = criterion(outputs_clean, labels)

            # Compute gradients with respect to inputs for adversarial example creation
            optimizer.zero_grad()
            loss_clean.backward(retain_graph=True)  # Keep the graph for the next backward pass
            inputs_grad = inputs.grad.data.clone()  # Clone the gradients to avoid them being overwritten

            # Zero the gradients for the next pass
            optimizer.zero_grad()

            # Generate adversarial examples using the perturbed inputs with collected gradients
            perturbed_inputs = fgsm_attack(inputs, epsilon, inputs_grad)

            # Forward pass on the adversarial examples and compute the loss
            outputs_adv = net(perturbed_inputs)
            loss_adv = criterion(outputs_adv, labels)  # Cross-entropy loss for adversarial examples

            # Calculate the classification loss on the clean outputs
            outputs_clean_detached = outputs_clean.detach()  # Detach the clean outputs
            loss_kl = F.kl_div(
                F.log_softmax(outputs_adv, dim=1),
                F.softmax(outputs_clean_detached, dim=1),
                reduction='batchmean'
            ) * beta  # Include the TRADES beta parameter

            # Combine the classification loss and regularization term into a single scalar
            loss_combined = loss_adv + loss_kl

            # Backward pass and optimize based on the combined loss
            loss_combined.backward()
            optimizer.step()

            if (i + 1) % 2000 == 0:
                print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(trainloader)}], Loss: {loss_combined.item():.4f}')




def train_PAT(net, trainloader, epsilon, alpha, epochs):
    net.train()

    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            inputs.requires_grad_(True)

            # Forward pass to calculate predictions and extract features
            outputs_clean, features_clean = net(inputs)
            loss_clean = criterion(outputs_clean, labels)

            # Calculate gradients for adversarial example generation
            loss_clean.backward(retain_graph=True)
            inputs_grad = inputs.grad.data
            inputs.grad.zero_()

            # Generate adversarial examples
            perturbed_inputs = fgsm_attack(inputs, epsilon, inputs_grad)

            # Forward pass with adversarial examples
            outputs_adv, features_adv = net(perturbed_inputs)
            loss_adv = criterion(outputs_adv, labels)

            # Calculate PAT loss (perceptual similarity term)
            perceptual_loss = F.pairwise_distance(features_clean, features_adv).mean()

            # Combined loss
            loss = loss_clean + alpha * perceptual_loss
            loss.backward()
            optimizer.step()

            if (i + 1) % 1000 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(trainloader)}], PAT Loss: {loss.item():.4f}')

def train_RoBal(net, trainloader, epsilon, epochs):
    net.train()

    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(epochs):
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)
            inputs.requires_grad_(True)

            # Calculate weighted loss on clean inputs
            outputs_clean = net(inputs)
            loss_clean = net.compute_loss(outputs_clean, labels)

            # Zero the gradients
            optimizer.zero_grad()

            # Calculate gradients for clean inputs
            loss_clean.backward(retain_graph=True)
            inputs_grad = inputs.grad.data

            # Zero the gradients after backward pass to start clean for adversarial example generation
            optimizer.zero_grad()

            # Generate adversarial examples
            perturbed_inputs = fgsm_attack(inputs, epsilon, inputs_grad)

            # Calculate weighted loss on adversarial examples
            outputs_adv = net(perturbed_inputs)
            loss_adv = net.compute_loss(outputs_adv, labels)

            # Combine losses
            loss_combined = loss_clean + loss_adv

            # Backward pass and optimize based on the combined loss
            loss_combined.backward()
            optimizer.step()

            if (i + 1) % 1000 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(trainloader)}], Loss: {loss_combined.item():.4f}')

In [6]:
# Define the epsilon for FGSM
epsilon = 0.1

# Training and testing on CIFAR-10
net_cifar10 = SimpleCNN(num_classes=10)
train(net_cifar10, trainloader_cifar10, epochs=1)
accuracy_cifar10 = test(net_cifar10, testloader_cifar10)
robustness_accuracy = evaluate_robustness(net_cifar10, testloader_cifar10, epsilon=epsilon)
print(f'Accuracy of the network on the CIFAR-10 test images: {accuracy_cifar10 * 100}%')
print(f'Accuracy on the adversarial set without adversarial training: {robustness_accuracy * 100: .2f}%')

[Epoch 1, 2000] loss: 2.132
[Epoch 1, 4000] loss: 1.803
[Epoch 1, 6000] loss: 1.612
[Epoch 1, 8000] loss: 1.521
[Epoch 1, 10000] loss: 1.478
[Epoch 1, 12000] loss: 1.417
Accuracy of the network on the CIFAR-10 test images: 50.09%
Accuracy on the adversarial set without adversarial training:  26.03%


In [7]:
# Train the network with adversarial examples
train_with_adversarial(net_cifar10, trainloader_cifar10, epsilon, epochs=1)

# Evaluate the accuracy and robustness of the trained model
accuracy = test(net_cifar10, testloader_cifar10)
robustness_accuracy = evaluate_robustness(net_cifar10, testloader_cifar10, epsilon=epsilon)

print(f'Accuracy on the test set: {accuracy * 100: .2f}%')
print(f'Accuracy on the adversarial set: {robustness_accuracy * 100: .2f}%')

[Epoch 1, Mini-batch 1000] loss: 1.326
[Epoch 1, Mini-batch 2000] loss: 1.178
[Epoch 1, Mini-batch 3000] loss: 1.806
[Epoch 1, Mini-batch 4000] loss: 1.927
[Epoch 1, Mini-batch 5000] loss: 1.648
[Epoch 1, Mini-batch 6000] loss: 1.488
[Epoch 1, Mini-batch 7000] loss: 3.133
[Epoch 1, Mini-batch 8000] loss: 2.493
[Epoch 1, Mini-batch 9000] loss: 3.194
[Epoch 1, Mini-batch 10000] loss: 5.941
[Epoch 1, Mini-batch 11000] loss: 4.743
[Epoch 1, Mini-batch 12000] loss: 8.086
Accuracy on the test set:  26.80%
Accuracy on the adversarial set:  37.46%


# Running on CIFAR-100 and ImageNet
Only run the following block if you want to train using the larger datasets.

## CIFAR-100 (Not used due to runtime constraints):
Similar in size to CIFAR-10 but with 100 classes containing 600 images each. It can also be obtained through torchvision.
## ImageNet (Not used due to runtime constraints):
A large visual database designed for use in visual object recognition research, with more than 14 million images categorized according to the WordNet hierarchy. ImageNet is not directly included in PyTorch's datasets, but it can be accessed following the instructions at http://www.image-net.org/.

Please note that due to the computational resources required, we limited our study to one epoch training on the CIFAR-10 dataset. The complete multi-epoch training across all datasets, including CIFAR-100 and ImageNet, is part of future work.

In [ ]:
# Training and testing on CIFAR-100
net_cifar100 = SimpleCNN(num_classes=100)
train(net_cifar100, trainloader_cifar100, epochs=1)
accuracy_cifar100 = test(net_cifar100, testloader_cifar100)
print(f'Accuracy of the network on the CIFAR-100 test images: {accuracy_cifar100 * 100}%')

# Training and testing on ImageNet (Placeholder)
net_imagenet = SimpleCNN(num_classes=1000)  # Assuming 1000 classes for ImageNet
train(net_imagenet, trainloader_imagenet, epochs=1)
accuracy_imagenet = test(net_imagenet, testloader_imagenet)
print(f'Accuracy of the network on the ImageNet test images: {accuracy_imagenet * 100}%')

# Training the individual CNNs
We will train the CNN instances with different adversarial methods.

In [8]:
# For TRADES training
epsilon = 0.1  # FGSM perturbation strength
beta = 1.0     # TRADES beta hyperparameter
epochs = 1     # Number of epochs to train for
train_TRADES(net_trades_cifar10, trainloader_cifar10, epsilon, beta, epochs)

# Testing the models' accuracy and robustness would be the same as before
accuracy_trades = test(net_trades_cifar10, testloader_cifar10)
print(f'Accuracy of TRADES on the test set: {accuracy_trades * 100: .2f}%')

robustness_accuracy_trades = evaluate_robustness(net_trades_cifar10, testloader_cifar10, epsilon=epsilon)
print(f'Accuracy of TRADES on the adversarial set: {robustness_accuracy_trades * 100: .2f}%')

Epoch [1/1], Step [2000/12500], Loss: 2.2979
Epoch [1/1], Step [4000/12500], Loss: 2.2732
Epoch [1/1], Step [6000/12500], Loss: 1.9542
Epoch [1/1], Step [8000/12500], Loss: 2.1338
Epoch [1/1], Step [10000/12500], Loss: 2.6021
Epoch [1/1], Step [12000/12500], Loss: 2.9948
Accuracy of TRADES on the test set:  14.44%
Accuracy of TRADES on the adversarial set:  17.91%


In [9]:
net_pat_cifar10 = PATCNN(num_classes=10).to(device)

# For PAT training
alpha = 1.5    # PAT alpha hyperparameter
train_PAT(net_pat_cifar10, trainloader_cifar10, epsilon, alpha, epochs)

accuracy_pat = test(net_pat_cifar10, testloader_cifar10)
print(f'Accuracy of PAT on CIFAR-10 test images: {accuracy_pat * 100:.2f}%')

robustness_accuracy_PAT = evaluate_robustness(net_pat_cifar10, testloader_cifar10, epsilon=epsilon)
print(f'Robustness of PAT on perturbed CIFAR-10 images: {robustness_accuracy_PAT * 100:.2f}%')


Epoch [1/1], Step [1000/12500], PAT Loss: 7.1570
Epoch [1/1], Step [2000/12500], PAT Loss: 6.0433
Epoch [1/1], Step [3000/12500], PAT Loss: 8.7731
Epoch [1/1], Step [4000/12500], PAT Loss: 7.0780
Epoch [1/1], Step [5000/12500], PAT Loss: 8.0055
Epoch [1/1], Step [6000/12500], PAT Loss: 6.6488
Epoch [1/1], Step [7000/12500], PAT Loss: 7.1328
Epoch [1/1], Step [8000/12500], PAT Loss: 5.6037
Epoch [1/1], Step [9000/12500], PAT Loss: 7.6323
Epoch [1/1], Step [10000/12500], PAT Loss: 8.2173
Epoch [1/1], Step [11000/12500], PAT Loss: 6.4723
Epoch [1/1], Step [12000/12500], PAT Loss: 3.9811
Accuracy of PAT on CIFAR-10 test images: 44.38%
Robustness of PAT on perturbed CIFAR-10 images: 22.36%


In [10]:
# Train RoBalCNN with adversarial examples
train_RoBal(net_robal_cifar10, trainloader_cifar10, epsilon, epochs)


accuracy_robal_cifar10 = test(net_robal_cifar10, testloader_cifar10)
print(f'Accuracy of RoBalCNN on CIFAR-10 test images: {accuracy_robal_cifar10 * 100:.2f}%')

robustness_robal_cifar10 = evaluate_robustness(net_robal_cifar10, testloader_cifar10, epsilon)
print(f'Robustness of RoBalCNN on perturbed CIFAR-10 images: {robustness_robal_cifar10 * 100:.2f}%')

Epoch [1/1], Step [1000/12500], Loss: 4.6588
Epoch [1/1], Step [2000/12500], Loss: 3.8909
Epoch [1/1], Step [3000/12500], Loss: 4.2004
Epoch [1/1], Step [4000/12500], Loss: 4.0785
Epoch [1/1], Step [5000/12500], Loss: 2.9427
Epoch [1/1], Step [6000/12500], Loss: 3.5142
Epoch [1/1], Step [7000/12500], Loss: 2.8668
Epoch [1/1], Step [8000/12500], Loss: 3.2341
Epoch [1/1], Step [9000/12500], Loss: 3.3158
Epoch [1/1], Step [10000/12500], Loss: 3.9952
Epoch [1/1], Step [11000/12500], Loss: 3.1770
Epoch [1/1], Step [12000/12500], Loss: 2.5586
Accuracy of RoBalCNN on CIFAR-10 test images: 45.48%
Robustness of RoBalCNN on perturbed CIFAR-10 images: 34.26%
